# Were to Rent an Apartment in Dublin?

## 1. Introduction: Defining the Business Problem <a name="introduction"></a>

#### 1.1 Background

Dublin, the capital and the largest city of Ireland, is situated on a bay of the North Atlantic Ocean and is bordered by the Dublin Mountains on the south. With an estimate of 50% of its citizens being younger than 25, Dublin is, for sure, one of the most youthful cities in Europe. Companies such as Google, Amazon, Microsoft, Citigroup, PayPal, Facebook, Twitter have European headquarters or operational bases there. As a consequence, Dublin is sometimes referred to as the new "Tech Capital of Europe".

First of all, Dublin is known for its many pubs across the city. When speaking about nightlife, the best known area among tourists is the Temple Bar Neighborhood. The area is also known for its street performers and small live music venues. Other areas like those around Leeson Street, Harcourt Street, South William Street or Camden Street are popular among locals. When speaking about cuisine, there were five restaurants in 2018 in Dublin which shared six Michelin stars: Restaurant Patrick Guilbaud had two stars and there were four other restaurants with one star each (according to the Michelin Guide). 

#### 1.2 The business problem 

Mr. John Guinness, a software developer, found a new job in Dublin and decided to move there. He is interested to rent an apartment. *Where are the best places for him to leave? Which districts in Dublin best fit his preferences?* 

We asked Mr. Guinness to provide a list of his preferences. Here it is: theaters, parks, beaches, gyms, public transportation, pubs, restaurants (especially Italian and Chinese), coffee shops, shopping malls, supermarkets, groceries and convenience shops.  

There are 22 districts in Dublin. In order to answer Mr. Guinness questions, we have to look for the most relevant venues for each district in Dublin, we have to group the districts regarding the categories of venues in Mr. Guinness' preferences list and, finally, we have to find the group of districts which best matches his preferences.

#### 1.3 Target audience

The target audience for this project could be anyone who looks for renting an apartment in Dublin. The project can be fitted to her/his list of preferences.

## 2. Data Acquisition and Preparation <a name="data"></a>

#### 2.1 Districts of Dublin: Building the data frame and rendering the map.

One can find a list of the districts and their corresponding postal codes on wikipedia:

https://en.wikipedia.org/wiki/List_of_Dublin_postal_districts

<img align="left" src="wikipedia.jpg" width="600">

We downloaded the file from wikipedia, we parsed it using the *html5lib* parser and we scraped it using the *BeautifulSoup* scraper. We looked for the table containing the information about the districts and we built a data frame containing the districts of Dublin, their corresponding postal codes and their descriptions.

Using the *Nominatim* geocoder from the *geopy* library we converted addresses into latitude and longitude values and we added the values to the data frame. After doing this, the data frame containing the districts of Dublin was complete. 

<img align="left" src="districts.jpg" width="600">

Using the *folium* library, we created a map of Dublin and we added to the map one marker for each district.

<img align="left" src="map_17.jpg" width="600">

There are 22 districts in the *dublin_districts* data frame, but we got markers for only 17 districts on the map. It means that 5 pair of coordinates (latitude and longitude) obtained using the *Nominatim* geocoder were incorrect. 

Looking to the values of latitudes and longitudes in our data frame, we found the incorrect values: those for districts D1, D2, D3, D17 and D24. For these districts we had to find the coordinates manually. We used the map and information from the Dublin postal code map from: https://www.irishtourist.com/tourist-information/getting-to-and-around-ireland/maps-of-ireland/dublin-postal-code-map/ and the google map of Dublin: https://www.google.com/maps/place/Dublin,+Ireland/@53.3244431,-6.385789,11z/

We found the following values:
* D1: 53.350062, -6.247473
* D2: 53.341753, -6.258112
* D3: 53.364304, -6.229960
* D17: 53.404862, -6.192409
* D24: 53.303134, -6.352392

After correction we got the following map:

<img align="left" src="map_22.jpg" width="600">

#### 2.2 Venues in the  districts of Dublin: building the data frame

In order to find the venues in the districts we used the *Foursquare* API. For each district in Dublin we built a GET request in order to get the venues (not more than 100) situated in a circle with a radius of 2 kilometers, centered in the latitude and longitude coordinates of the district. 

We created a data frame which contains all the venues in the districts of Dublin. Each venue is a row in our data frame and is characterized by the following features: name of district, latitude of district, longitude of district, name of venue, latitude of venue, longitude of venue and category of venue.

## 3. Methodology <a name="methodology"></a>

#### 3.1 Exploratory data analysis: Analyzing a district in Dublin

First, we explored a central district in Dublin, the one with the D1 postal code. From the data frame containing information about the districts in Dublin, we got the latitude and longitude values for the D1 district: 53.350062 and -6.247473. In order to get the venues (not more than 100) situated in a circle with a radius of 2 kilometers and centered in the point defined by these values, we built a GET request to the Foursquare API. 

**_Note: The data used in our model was received from Foursquare API on May 19, around 15:40, Dublin time._**

Analyzing the results of the request in *jason* format, accessed by *requests.get(url).json()*, we found that the information about each venue can be found in the *items* key. Accessing it, we got a list of venues for the D1 district.

We structured the *json* format list into a *pandas* data frame and filtered the columns regarding to venue name, venue category, venue latitude and venue longitude. 

In order to understand the data in the venue categories field, we needed to define a function to look for the eventual different categories for each venue, or the eventual (not desired) absence of data for this field. Analyzing the results we understood that for each venue we got only one category and no data was missing. Later on, we used this conclusion when we looked for the categories of venues for all the districts in Dublin.

At this stage, we got a data frame containing 100 venues in the D1 district of Dublin. Each venue is a row in the data frame and is characterized by the following features: name of venue, category of venue, latitude of venue and longitude of venue.

<img align="left" src="table_1.jpg" width="450">

#### 3.2 Inferential statistical testing: Analyzing all the districts in Dublin

Using the paradigm from the chapter 3.1, we defined a function in order to repeat the same process for all the districts in Dublin. Considering the conclusion above - for each venue we got only one category and no data was missing -, we got the category of each venue by accessing the element in the list of venues.

Using this function, we created a data frame which contains all the venues in the districts of Dublin. Each venue is a row in our data frame and is characterized by the following features: name of district, district latitude, district longitude, name of venue, venue latitude, venue longitude and venue category.

<img align="left" src="table_2.jpg">

Now, we could easily got the total number of venues in our data frame, the number of venues in each district, or the number of unique categories of venues.

Applying the *get_dummies* function from the *pandas* library on the venue category column of the data frame, we transformed the data frame into a new one containing numerical data. We grouped rows by district and by taking the mean of the frequency of occurrence of each category of venues. We got a new data frame, *dublin_grouped*. This data frame, or, more exactly, part of it, was used later for the clustering process. 

In order to better understand our data, we defined a function to sort the categories of venues in descending order regarding the frequency of occurrence of each category and built a new data frame containing each district along with the top most common venues.

<img align="left" src="table_3.jpg">

Based on the information in this data frame and considering Mr. Guinness' preferences, we extracted from the *dublin_grouped* data frame the relevant columns (= features = district name + categories of venues): *'District', 'Pub', 'Café', 'Coffee Shop', 'Park', 'Beach', 'Gym', 'Theater', 'Supermarket', 'Grocery Store', 'Convenience Store', 'Shopping Mall', 'Chinese Restaurant', 'Italian Restaurant', 'Light Rail Station', 'Bus Stop'*. The new data frame was named *dublin_grouped_for_mr_guinness*.

<img align="left" src="table_5.jpg">

Its subset containing only numerical data (the frequencies of occurrence of each venue category) was used for the clustering process.

#### 3.3 Machine learning technics: Clustering the districts in Dublin

##### 3.3.1 The algorithm: Agglomerative Hierarchical Clustering  

Hierarchical Clustering algorithms build a hierarchy of clusters where each node is a cluster which consists of the clusters of its daughter nodes. The agglomerative clustering algorithm performs a hierarchical clustering using a bottom up approach: pairs of clusters are merged together as they move up the hierarchy.

Having a data set with n data points, the algorithm starts by assigning each data point to its own cluster. The next step is to look for the closest two clusters one to another. 

We construct a distance matrix where the ij element in the matrix is the distance between the i and j data points, with i,j = 1,...,n. The distance matrix contains the distances from each point to every other point of the dataset.   

At each iteration, the algorithm updates the distance matrix to reflect the distance of the newly formed cluster with the remaining clusters in the set. 

The steps of the algorithm are as follows:
* create n clusters and assign one for each data point. 
* compute the distance matrix.
* repeat:
  * merge the two closest clusters.
  * update the distance matrix.
* until only a single cluster remains.

There are different approaches to calculate the distance between clusters and they define different  paradigms:
* Single linkage clustering, where the minimum distance between data points in the two clusters is calculated.
* Complete linkage clustering, where the maximum distance between data points in the two clusters is calculated.
* Average linkage clustering: where the average distance between data points in the two clusters is calculated.
* Centroid linkage clustering: where the distance between the two clusters centroids is calculated.

The algorithm can be visualized building a dendrogram. Each merge of clusters is a horizontal line in the dendrogram and the *y* coordinate of the horizontal line represents the similarity of the two clusters that were merged. The dendrogram allows us to reconstruct the history of merges in the clustering process. 

Why did we chose the agglomerative hierarchical clustering algorithm? First of all, the algorithm produces a dendrogram which can help to better understand the data and how the clusters were built. Second, our data set is a small one, therefore we do not need a more efficient algorithm. Finally, the algorithm always generates the same clusters.

##### 3.2.2 Clustering the districts

As we have already described above, the *dublin_grouped_for_mr_guinness* data frame contains the relevant categories of venues based on Mr. Guinness' preferences. Its subset containing only the numerical data - the frequencies of occurrence of each venue category , was used as input for the clustering process.

In order to understand the clustering process we built its dendrogram using the distance matrix:

<img align="left" src="dendrogram.jpg" width="360">

Essentially, the agglomerative hierarchical clustering algorithm does not require a pre-specified number of clusters. However, in our applications we want to choose the most relevant number of clusters for our purpose. Analyzing the dendrogram we concluded that a good choice for the number of clusters is n_clusters = 5.

We passed two inputs to the agglomerative clustering algorithm: the number of clusters to be formed (n_clusters = 5), and the paradigm to be used (linkage = 'complete'), the complete linkage clustering, where the maximum distance between data points in the clusters is calculated.

## 4. Results <a name="results"></a>

Five clusters have been generated by the agglomerative hierarchical clustering algorithm:

<img align="left" src="map_clusters.jpg" width="600">

#### Cluster 1:

<img align="left" src="cluster_1.jpg">

The most relevant venue categories for **_cluster 1_** are: **_supermarkets, parks, shopping malls, grocery stores, coffee shops_** and **_pubs_**.

#### Cluster 2:

<img align="left" src="cluster_2.jpg">

The most relevant venue categories for **_cluster 2_** are: **_pubs, cafés, grocery stores, bus stops, Italian restaurants_** and **_supermarkets_**.

#### Cluster 3:

<img align="left" src="cluster_3.jpg">

The most relevant venue categories for **_cluster 3_** are: **_convenience stores, bus stops, Italian restaurants, grocery stores, pubs_** and **_light rail stations_**.

#### Cluster 4:

<img align="left" src="cluster_4.jpg">

The most relevant venue categories for **_cluster 4_** are: **_cafés, coffee shops, pubs, Italian restaurants, parks_** and **_grocery stores_**.

#### Cluster 5:

<img align="left" src="cluster_5.jpg">

##### The most relevant venue categories for **_cluster 5_** are: **_supermarkets, coffee shops, pubs, cafés, gyms_** and **_parks_**.

## 5. Discussion <a name="discussion"></a>

#### 5.1 Observations

For each district, the ten most relevant categories of venues were considered in order to describe it. Each cluster was described by the six most relevant venue categories among all its member districts.

We can observe that the most relevant venue categories for **_cluster 1_** are: **_supermarkets, parks, shopping malls, grocery stores, coffee shops_** and **_pubs_**. **_Supermarkets_** are on the first position in five of the six districts in this cluster and on the second position for the 6-th district. Three districts have all the six venue categories which describe this cluster, the other three districts have five of these categories.

For **_cluster 2_**, the most relevant venue categories are: **_pubs, cafés, grocery stores, bus stops, Italian restaurants_** and **_supermarkets_**. **_Pubs_** are on the first position in two of the three districts in this cluster and on the second position for the third district. Two districts have all the six venue categories which describe this cluster, the third district have five of these categories.

**_Cluster 3_** contains only one district and the most relevant venue categories for this district are: **_convenience stores, bus stops, Italian restaurants, grocery stores, pubs_** and **_light rail stations_**. 

For **_cluster 4_**, the most relevant venue categories are: **_cafés, coffee shops, pubs, Italian restaurants, parks_** and **_grocery stores_**. **_Cafés_** are on the first position in five of the eight districts in this cluster, on the second position in two districts and on the third position in the eighth district. **_Cluster 4 is a very compact one_**: Not only that all the districts have all six venue categories which describe this cluster, but **_these venue categories are among the eight most relevant ones in each district_**. 

The results for **_cluster 4_** are, actually, the expected ones. As we can see on the map, the districts in this cluster are also **_spatially clustered_**: they are the central districts of Dublin. 

For **_cluster 5_**, the most relevant venue categories are: **_supermarkets, coffee shops, pubs, cafés, gyms_** and **_parks_**. **_Supermarkets_** are on the first position in three of the four districts in this cluster. All the districts have all the venue categories which describe this cluster.

The table below summarize the information presented above and was presented to Mr. Guinness:


| cluster | no. of districts   | first cat.         | second cat.         | third cat.          | fourth cat.         | fifth cat.          | sixth cat.          |
|:--------|:-------------------|:-------------------|:--------------------|:--------------------|:--------------------|:--------------------|:--------------------|
| 1       | 6                  | supermarkets       | parks               | shopping malls      | grocery stores       | coffee shops        | pubs                |
| 2       | 3                  | pubs               | cafés               | grocery stores      | bus stops           | Italian restaurants | supermarkets        |
| 3       | 1                  | convenience stores | bus stops           | Italian restaurants | grocery stores       | pubs                | light rail stations |
| 4       | 8                  | cafés              | coffee shops        | pubs                | Italian restaurants | parks               | grocery stores      |
| 5       | 4                  | supermarkets       | coffee shops        | pubs                | cafés               | gyms                | parks               |

#### 5.2 Recommendations

Based on the results described in the previous paragraph, we can recommend to Mr. Guinness the following: 
* If he prefers a central district with cafés, coffee shops, pubs, Italian restaurants and parks, one of the eight districts in **_cluster 4_** will be the best choice.
* If he prefers supermarkets, parks, shopping malls, coffee shops and pubs, one of the six districts in **_cluster 1_** could be a good choice.
* If his first preferences are supermarkets, coffee shops, pubs, cafés, gyms and parks, then one of the four districts in **_cluster 5_** will be a good choice.
* One of the three districts in **_cluster 2_** could be a good choice if he prefers pubs, cafés, Italian restaurants, supermarkets and is interested in public transportation.
* The single district in **_cluster 5_** seems to be a poor choice. It contains convenience stores, bus stops, Italian restaurants, grocery stores, pubs and light rail stations.

## 6. Conclusion <a name="conclusion"></a>

Mr. Guinness is interested to rent an apartment in Dublin. The target of this project was to identify the districts in Dublin which best fit Mr. Guinness' preferences.

A data frame, containing the districts of Dublin, their corresponding postal codes and their descriptions was build using a file downloaded from wikipedia. Latitude and longitude values for each district were added to the data frame. 

A second data frame containing venues in the districts of Dublin was built using the *Foursquare* API. 
The venues were grouped by district and by taking the mean of the frequency of occurrence of each category of venues. Considering Mr. Guinness' preferences, the relevant columns were extracted from this data frame. 

Using this data frame, the agglomerative clustering algorithm, which performs a hierarchical clustering using a bottom up approach, was used in order to group the districts in Dublin.

A summary of the results was presented to Mr. Guinness along with our recommendations. 

As future work, we consider to develop the presented model by adding information regarding rent levels for apartments in different districts in Dublin. 